In [1]:
import json
import pymysql

from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from bs4 import BeautifulSoup

import re

import pandas as pd

In [2]:
# 크롬 드라이버 설정
global driver
driver = webdriver.Chrome()
global wait
wait = WebDriverWait(driver, 20)

# User-Agent 설정
user_agent = 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/110.0.0.0 Safari/537.36'
options = webdriver.ChromeOptions()
options.add_argument('user-agent=' + user_agent)

In [3]:
with open('info.json', 'r') as file:
    data = json.load(file)
    db_pw = data['db_info']['pw']

    id = data['user_info']['id']
    pw = data['user_info']['pw']

In [4]:
def __init__(db_pw):
    conn = pymysql.connect(host='127.0.0.1', user='root', password=db_pw, db='baekjoon', charset='utf8')

    with conn.cursor() as curs:
        sql="""
        CREATE TABLE IF NOT EXISTS problem_info(
            number INT,
            title VARCHAR(40),
            isSolved BOOLEAN,
            level INT,
            PRIMARY KEY (number)
        );

        """
        curs.execute(sql)

    conn.commit()

In [5]:
def login_solved(id, pw):
    '''
    solved 웹사이트에 접속해서 계정으로 로그인한다.
    '''
    login_url = "https://www.acmicpc.net/login?next=%2Fsso%3Fsso%3Dbm9uY2U9YTM5YWVhYTk4MWNhYjU1YWEwZDRlZDQ4Nzc3NzEzOGM%253D%26sig%3D1ca260b9abbbe4df2b002254dd94a7f31b373ff0270b243408963284f8fcd7a1%26redirect%3Dhttps%253A%252F%252Fsolved.ac%252Fapi%252Fv3%252Fauth%252Fsso%253Fprev%253D%25252F"
    driver.get(login_url)

    # 로그인 페이지에서 계정 정보를 입력하고 클릭한다.
    wait.until(
        EC.presence_of_all_elements_located((By.XPATH, '//*[@id="login_form"]/div[2]/input'))
    )

    user_id = driver.find_element(By.XPATH, '//*[@id="login_form"]/div[2]/input')
    user_pw = driver.find_element(By.XPATH, '//*[@id="login_form"]/div[3]/input')

    user_id.send_keys(id)
    user_pw.send_keys(pw)

    login_bt = driver.find_element(By.XPATH, '//*[@id="submit_button"]')
    login_bt.click()  

    # 백준 계정으로 솔브드에 로그인할지 묻는 화면. 수락을 클릭함
    wait.until(
        EC.presence_of_all_elements_located((By.XPATH, '//*[@id="login_form"]/div[4]/div[2]/a'))
    )

    login_bt_2 = driver.find_element(By.XPATH, '//*[@id="login_form"]/div[4]/div[2]/a')
    login_bt_2.click()

In [11]:
def read_solved():
    '''
    솔브드 레벨을 순회하며 문제 정보를 스크래핑한다.
    '''
    def has_korean(text):
        '''
        텍스트에 한글을 포함하고 있는지 확인하는 코드
        ''' 
        # 한글 유니코드 범위: AC00-D7AF
        korean_regex = re.compile("[\uac00-\ud7af]+")
        return bool(korean_regex.search(text))


    max_level = 2 # 골드 1
    df = pd.DataFrame(columns=['number', 'title','isSolved', 'level'])
    for level in range(1, max_level+1):
        # 각 레벨의 1페이지에 접속하고 html 텍스트 정보를 파싱함
        url = f'https://solved.ac/problems/level/{level}'
        driver.get(url)
        html = BeautifulSoup(driver.page_source, 'lxml')

        # 마지막 페이지 번호 추출
        last_page = int(html.find_all('a', {'class':'css-1yjorof'})[-1].text)

        # 1페이지의 데이터를 수집함.
        

        rows = html.find_all('tr', {'class':'css-1ojb0xa'})

        for row in range(1, len(rows)):
            info = rows[row].find_all('a', {'class':'css-q9j30p'})
            number = int(info[0].find('span').text)
            title = info[1].find('span', {'class':'__Latex__'}).text

            korean = has_korean(title)

            #한국어로 된 문제가 아니라면 저장하지 않고 패스한다.
            if korean == False:  
                continue
            isSolved = True if rows[row].find_all('span', {'class':'ac'}) else False
            title = re.sub("'", '', title) # 따옴포를 제거함.

            #데이터 삽입
            df.loc[len(df)] = [number, title, isSolved, level]
            print(df.loc[len(df)-1])


        # 2페이지부터 순회하며 데이터 수집
        for page in range(2, last_page + 1):
            pg_url = f'?page={page}'
            driver.get(url+pg_url)
            html = BeautifulSoup(driver.page_source, 'lxml')

            rows = html.find_all('tr', {'class':'css-1ojb0xa'})

            for row in range(1, len(rows)):
                info = rows[row].find_all('a', {'class':'css-q9j30p'})
                number = int(info[0].find('span').text)
                title = info[1].find('span', {'class':'__Latex__'}).text

                korean = has_korean(title)

                #한국어로 된 문제가 아니라면 저장하지 않고 패스한다.
                if korean == False: 
                    continue
                isSolved = True if rows[row].find_all('span', {'class':'ac'}) else False
                title = re.sub("'", '', title) # 따옴포를 제거함.

                #데이터 삽입
                df.loc[len(df)] = [number, title, isSolved, level]
                print(df.loc[len(df)-1])
    print("백준 문제 크롤링 완료")            
    return df

In [7]:
def insert_into_db(df):
    '''
    데이터베이스의 데이터를 입력한다.
    '''
    with conn.cursor() as curs:
        for r in df.itertuples(index=False):
            print(r)
            sql=f"INSERT INTO problem_info VALUES ({r.number}, '{r.title}', {r.isSolved}, {r.level})"
            curs.execute(sql)

        conn.commit()
    print(f"[{len(df)}]개의 데이터 DB 저장 완료")

In [8]:
__init__(db_pw)

In [9]:
login_solved(id, pw)

In [12]:
df=read_solved()

number         1271
title       엄청난 부자2
isSolved       True
level             1
Name: 0, dtype: object
number          1330
title       두 수 비교하기
isSolved        True
level              1
Name: 1, dtype: object
number        2338
title       긴자리 계산
isSolved      True
level            1
Name: 2, dtype: object
number       2420
title       사파리월드
isSolved     True
level           1
Name: 3, dtype: object
number          2438
title       별 찍기 - 1
isSolved        True
level              1
Name: 4, dtype: object
number      2475
title        검증수
isSolved    True
level          1
Name: 5, dtype: object
number       2738
title       행렬 덧셈
isSolved     True
level           1
Name: 6, dtype: object
number      2739
title        구구단
isSolved    True
level          1
Name: 7, dtype: object
number      2741
title       N 찍기
isSolved    True
level          1
Name: 8, dtype: object
number          2743
title       단어 길이 재기
isSolved       False
level              1
Name: 9, dtype: object
number        

In [13]:
df

,number,title,isSolved,level
0,1264,모음의 개수,False,2
1,2083,럭비 클럽,False,2
2,2439,별 찍기 - 2,True,2
3,2440,별 찍기 - 3,True,2
4,2480,주사위 세개,False,2
...,...,...,...,...
61,25628,햄버거 만들기,False,2
62,25703,포인터 공부,False,2
63,25704,출석 이벤트,False,2
64,26068,치킨댄스를 추는 곰곰이를 본 임스 2,False,2


In [14]:
conn.close()

In [16]:

conn = pymysql.connect(host='127.0.0.1', user='root', password=db_pw, db='baekjoon', charset='utf8')
with conn.cursor() as curs:
    for r in df.itertuples(index=False):
        print(r.number, r.title, r.isSolved, r.level)
        sql=f"INSERT INTO problem_info VALUES ({r.number}, '{r.title}', {r.isSolved}, {r.level})"
        # sql=f"INSERT INTO problem_info VALUES (%s, '%s', %s, %s, %s)" %(r.number, r.title, r.isSolved, r.level, r.korean)
        curs.execute(sql)
    conn.commit()

1264 모음의 개수 False 2
2083 럭비 클럽 False 2
2439 별 찍기 - 2 True 2
2440 별 찍기 - 3 True 2
2480 주사위 세개 False 2
2530 인공지능 시계 False 2
2742 기찍 N True 2
2752 세수정렬 False 2
2845 파티가 끝나고 난 뒤 True 2
4299 AFC 윔블던 False 2
4470 줄번호 False 2
5524 입실 관리 False 2
5532 방학 숙제 False 2
5543 상근날드 True 2
5554 심부름 가는 길 False 2
5575 타임 카드 False 2
5596 시험 점수 False 2
5717 상근이의 친구들 False 2
10039 평균 점수 True 2
10101 삼각형 외우기 False 2
10156 과자 False 2
10768 특별한 날 False 2
10797 10부제 False 2
10808 알파벳 개수 False 2
11282 한글 False 2
11283 한글 2 False 2
11365 !밀비 급일 False 2
11720 숫자의 합 True 2
11943 파일 옮기기 False 2
11945 뜨거운 붕어빵 False 2
11948 과목선택 False 2
13118 뉴턴과 사과 False 2
13866 팀 나누기 False 2
14470 전자레인지 False 2
14489 치킨 두 마리 (...) False 2
14924 폰 노이만과 파리 False 2
15552 빠른 A+B True 2
15680 연세대학교 False 2
15700 타일 채우기 4 False 2
15726 이칙연산 False 2
15873 공백 없는 A+B True 2
15921 수찬은 마린보이야!! False 2
16199 나이 계산하기 False 2
16204 카드 뽑기 False 2
16486 운동장 한 바퀴 False 2
17356 욱 제 False 2
17362 수학은 체육과목 입니다 2 False 2
17388 와글와글 숭고한 False 2
19698 헛간 